In [8]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9150)
socket.socket = socks.socksocket
import requests
print (requests.get('http://icanhazip.com')).content

import json
import csv
import re
import sys 
import requests
reload(sys)
sys.setdefaultencoding('UTF-8')
import string
import time

t0=time.time()
#Definitions
global commsite, start, end,headers,headersReq
commsite = 'http://www.dailymail.co.uk/reader-comments/p/asset/readcomments/'
start = '?max=100&offset='
end ='&order=desc&rcCache=shout'
headers = 'id', 'dateCreated', 'message', 'assetId', 'assetUrl','assetCommentCount', 'userAlias','userLocation','assetHeadline',   'userIdentifier', 'voteRating', 'voteCount', 'hasProfilePicture'
headings = 'Comment ID', 'Comment Created Time', 'Text', 'Article ID', 'Article URL','assetCommentCount', 'User Alias','User Location','Article Headline',   'User ID', 'voteRating', 'voteCount', 'hasProfilePicture'
headersReq = {
    'Host': 'www.dailymail.co.uk',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
}


#Precompile RegExs
timeRegex = re.compile(r"(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):.+?Z")
pubRegex = re.compile(r"<meta property =\"article:published_time\" content=\"(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):.+?")

n=2

#UI
#URL = raw_input('Enter Full Daily Mail Article URL: ')
#article = re.search('\/article-(\d+)\/',URL).group(1)

#Setup CSV
outfile  = open('DMData.csv', "wb")
writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
writer.writerow(headings+('length','artCreatedDate','Elapsed Time (minutes)','AverageWordCount','FacebookURL','Number of Parent Comments'))

#Isolate post ids from data.csv
fb_ids = []
fb_idss= ['1234','1367474646645613', '1368068083252936', '1367411916651886', '1366269283432816']

with open('data.csv','rb') as csvfile:
    reader = csv.DictReader (csvfile)
    for row in reader:
        if row['is_post']=='1':
            match = re.search('._(.+)',row['id'])
            fb_ids.append(match.group(1))

def scrape(sec2,art2):
#Find Publish date
    global firstLine, currentLine, lastLine
    #print 'inside scrape loop'
    FacebookURL = [fb]
    URL = 'http://www.dailymail.co.uk/' + sec2 + '/article-' + art2
    req = requests.get(URL,headers=headersReq)
    lines = req.text
    match = re.search(pubRegex,lines)
    if match:
        pubtime=match.groups()
    pubTime = [match.group(3)+'/'+match.group(2)+'/'+match.group(1)+' '+match.group(4)+':'+match.group(5)]

    #First set of comments
    reqURL = commsite + art2
    req = requests.get (reqURL, headers = headersReq)
    html_source = req.text

    numOfComments = re.findall ('{"total":(\d+)',html_source)
    numOfParents = re.findall (',\"parentCommentsCount":(\d+),',html_source)
    nums = int(numOfComments[0])
    nump = int(numOfParents[0])
    numpList=[nump]
    
    lastLine = firstLine + nump-1

    #Get rest of comments
    messagesLen=[]
    #n=exlCounter
    for i in range (0, nums,100): 
        comment = []
        req = requests.get (reqURL + start + str(i) + end, headers = headersReq)
        data = json.loads(req.text)
        for i in range(len(data['payload']['page'])):
            for header in headers:
                if header == 'dateCreated':
                    match = re.search(timeRegex, (data['payload']['page'][i]['dateCreated']))
                    if match:
                        (data['payload']['page'][i]['dateCreated']) = match.group(3)+'/'+match.group(2)+'/'+match.group(1)+' '+match.group(4)+':'+match.group(5)
                comment.append(data['payload']['page'][i].get(header,'N\\A'))
                if header == 'message':
                    words = str(data['payload']['page'][i]['message'])
                    words = words.translate(None, string.punctuation)
                    words = re.split(r'[^0-9A-Za-z]+',words.strip())
                    wordsCount=str(len(words)).split()
                    messagesLen.append(len(words))
                delta = []
            #print 'before delta'
            delta = ['=' + '(' + 'B' + str(currentLine) + '-' + 'O' + str(currentLine) + ')' + '*1440'] #MS Excel format
            #print 'before avg'
            averageWordCount = ['=average($N$'+str(firstLine) + ':' + 'N' + str(lastLine) + ')']
            #print 'after avg'
            comment=comment+wordsCount+pubTime+delta + averageWordCount+FacebookURL+numpList
            writer.writerow(comment)
            comment = []
            currentLine+=1
    firstLine = lastLine +1
    #exlCounter+= nump
            
firstLine = 2 #initiate excel counter for multifile extaction in one csv
currentLine = 2
lastLine = 2

counter = 1
successes = []
fails= []
for id in fb_ids:
    try:
        fb = 'https://www.facebook.com/DailyMail/posts/'+ id #str(1367031010023310)
        #print 'Downloading ' + fb
        req = requests.get(fb) #get DM post on fb  
        #print 'Searching daim.ai URL'
        match = re.search ('https:\/\/l\.facebook.com\/l\.php\?u\=http%3A%2F%2Fdailym\.ai\%2F(.+?)\&amp;',req.text) #get DM minisite article number from fb
        #print 'dailm.ai match found'
        article1 = 'http://dailym.ai/' + match.group(1)
        req = requests.get(article1)
        #print 'Got dail.ai article'
        match = re.search ('<meta content=\"http:\/\/www\.dailymail\.co\.uk\/(.+)\/article-(.+?)\/',req.text) #get full article url
        #print 'Seraching for dailymail.co.uk URL'
        sec2 = match.group(1)
        art2 = match.group(2)

        scrape (sec2,art2)
        print 'Done with id# ' + id + '.\t Status: SUCCESS. \t(',counter,'out of ' + str(len(fb_ids)) + ')'
        counter+=1
        successes.append(id)
#        print (requests.get('http://icanhazip.com')).content

    except:
        print 'Done with id# ' + id + '.\t Status: FAIL. \t(',counter,'out of ' + str(len(fb_ids)) + ')'
        counter+=1
        fails.append(id)
#        print (requests.get('http://icanhazip.com')).content
    
outfile.close()

req.connection.close()

t1=time.time()